In [1]:
import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from IPython.display import Audio

c:\Users\wb_yuxuan.qi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Models

In [2]:
chat = ChatTTS.Chat()
chat.load_models()

# Use force_redownload=True if the weights updated.
# chat.load_models(force_redownload=True)

# If you download the weights manually, set source='locals'.
# chat.load_models(source='local', local_path='YOUR LOCAL PATH')

INFO:ChatTTS.core:Load from cache: C:\Users\wb_yuxuan.qi/.cache/huggingface\hub/models--2Noise--ChatTTS/snapshots\c0aa9139945a4d7bb1c84f07785db576f2bb1bfa
INFO:ChatTTS.core:use cpu
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.


## Inference

### Batch infer

In [3]:
texts = ["So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with.",]*3 \
        + ["我觉得像我们这些写程序的人，他，我觉得多多少少可能会对开源有一种情怀在吧我觉得开源是一个很好的形式。现在其实最先进的技术掌握在一些公司的手里的话，就他们并不会轻易的开放给所有的人用。"]*3     
        
wavs = chat.infer(texts)

INFO:ChatTTS.core:All initialized.
  0%|          | 0/384 [00:00<?, ?it/s]W0531 15:20:44.317000 1224 torch\_dynamo\convert_frame.py:824] WON'T CONVERT _update_causal_mask c:\Users\wb_yuxuan.qi\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\llama\modeling_llama.py line 1005 
W0531 15:20:44.317000 1224 torch\_dynamo\convert_frame.py:824] due to: 
W0531 15:20:44.317000 1224 torch\_dynamo\convert_frame.py:824] Traceback (most recent call last):
W0531 15:20:44.317000 1224 torch\_dynamo\convert_frame.py:824]   File "c:\Users\wb_yuxuan.qi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_dynamo\convert_frame.py", line 786, in _convert_frame
W0531 15:20:44.317000 1224 torch\_dynamo\convert_frame.py:824]     result = inner_convert(
W0531 15:20:44.317000 1224 torch\_dynamo\convert_frame.py:824]              ^^^^^^^^^^^^^^
W0531 15:20:44.317000 1224 torch\_dynamo\convert_frame.py:824]   File "c:\Users\wb_yuxuan.qi\AppData\Local\Programs\Python\Python3

In [4]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [5]:
Audio(wavs[3], rate=24_000, autoplay=True)

### Custom params

In [6]:
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}

wav = chat.infer('四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
  0%|          | 1/384 [00:04<27:27,  4.30s/it]W0531 15:22:52.791000 1224 torch\_dynamo\convert_frame.py:357] torch._dynamo hit config.accumulated_cache_size_limit (64)
W0531 15:22:52.791000 1224 torch\_dynamo\convert_frame.py:357]    function: 'forward' (c:\Users\wb_yuxuan.qi\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\llama\modeling_llama.py:587)
W0531 15:22:52.791000 1224 torch\_dynamo\convert_frame.py:357]    last reason: ___check_obj_id(L['self'], 2765207474768)                   
W0531 15:22:52.791000 1224 torch\_dynamo\convert_frame.py:357] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0531 15:22:52.791000 1224 torch\_dynamo\convert_frame.py:357] To diagnose recompilation issues, see https://pytorch.org/docs/master/compile/troubleshooting.html.
 20%|██        | 410/2048 [00:20<01:20, 20.24it/s]


In [7]:
Audio(wav[0], rate=24_000, autoplay=True)

### fix random speaker

In [8]:
rand_spk = chat.sample_random_speaker()
params_infer_code = {'spk_emb' : rand_spk, }

wav = chat.infer('四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 28%|██▊       | 570/2048 [00:28<01:12, 20.33it/s]


In [11]:
Audio(wav[0], rate=24_000, autoplay=True)

### Two stage control

In [12]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
chat.infer(text, refine_text_only=True)

INFO:ChatTTS.core:All initialized.
 21%|██        | 81/384 [00:04<00:17, 17.30it/s]


['so [uv_break] we found being competitive and collaborative was a huge way of staying motivated towards our goals, so [uv_break] one person to call when you fall off, [uv_break] one person who gets you back on then [uv_break] one person to actually do the activity with.']

In [13]:
text = 'so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.'
wav = chat.infer(text, skip_refine_text=True)

INFO:ChatTTS.core:All initialized.
 46%|████▌     | 938/2048 [00:48<00:57, 19.35it/s]


## LLM Call

In [9]:
from ChatTTS.experimental.llm import llm_api

API_KEY = 'sk-ce944028173941d6bbfa9eb30acf9110'
client = llm_api(api_key=API_KEY,
        base_url="https://api.deepseek.com",
        model="deepseek-chat")

In [10]:
user_question = '四川有哪些好吃的美食呢?'
text = client.call(user_question, prompt_version = 'deepseek')
print(text)
text = client.call(text, prompt_version = 'deepseek_TN')
print(text)

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了, 有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、夫妻肺片、回锅肉、串串香、宜宾燃面、乐山钵钵鸡、绵阳米粉等等。四川菜以麻辣鲜香著称, 口味独特, 让人回味无穷。


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、夫妻肺片、回锅肉、串串香、宜宾燃面、乐山钵钵鸡、绵阳米粉等等。四川菜以麻辣鲜香著称，口味独特，让人回味无穷。


In [14]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}

wav = chat.infer(text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 48%|████▊     | 985/2048 [00:50<00:54, 19.58it/s]
